In this file we define the classes (objects) used by all of the systems

In [1]:
class Case():
    # a case is defined as a triple Case(name, fact situtation, outcome)
    def __init__(self, name, fs, outcome):
        self.name = name
        self.fs = fs
        self.outcome = outcome
        
        # for convenience we add the attribute opposite_outcome
        self.opposite_outcome = abs(self.outcome - 1)
        
    def print_case(self):
        print(self.name)
        for feature, value in self.pro():
            print('Pro: ', feature.name)
        for feature, value in self.con():
            print('Con :', feature.name)        
        for feature, value in self.pro_dim():
            print('Pro_dim:', feature.name, " : ", value)
        for feature, value in self.con_dim():
            print('Con_dim:', feature.name, " : ", value)
        print('Outcome: ', self.outcome)
        
    # given a case, returns pro factors
    def pro(self):
        pro = []
        for feature, value in self.fs.items():
            if feature.factor:
                if feature.get_tendency(value) == self.outcome:
                    pro.append((feature, value))
        return pro
    
    # given a case, returns con factors
    def con(self):
        con = []
        for feature, value in self.fs.items():
            if feature.factor:
                if feature.get_tendency(value) == self.opposite_outcome:
                    con.append((feature, value))
        return con
    
    # given a case, returns pro dimensions (this term is not used by Prakken)
    # with pro dimension we mean a dimension on which a higher value promotes the own outcome
    def pro_dim(self):
        pro_dim = []
        for feature, value in self.fs.items():
            if not feature.factor:
                if feature.get_tendency(value) == self.outcome:
                    pro_dim.append((feature, value))
        return pro_dim
    
    # given a case, returns con dimensions (this term is not used by Prakken)
    # with con dimension we mean a dimension on which a higher value promotes the opposite outcome
    def con_dim(self):
        con_dim = []
        for feature, value in self.fs.items():
            if not feature.factor:
                if feature.get_tendency(value) == self.opposite_outcome:
                    con_dim.append((feature, value))
        return con_dim

    # given a case and another case, returns a comparison object
    # a comparison object collects all relevant and all other differences separately
    def compare(self, case):

        # relevant differences
        missing_pro = [p for p in case.pro() if p not in self.pro()]
        new_con = [c for c in self.con() if c not in case.con()]
        
        # other differences (these terms are not used in Prakken)
        new_pro = [p for p in self.pro() if p not in case.pro()]
        missing_con = [c for c in case.con() if c not in self.con()]
        
        worse, better = [], []
        for feature, value in self.pro_dim():
            # relevant difference
            if case.fs[feature] > value:
                worse.append((feature, value))
            # other difference
            elif case.fs[feature] < value:
                better.append((feature, value))
                
        for feature, value in self.con_dim():
            # relevant difference
            if case.fs[feature] < value:
                worse.append((feature, value))
            # other difference
            elif case.fs[feature] > value:
                better.append((feature, value))
        
        return Comparison(missing_pro, new_con, worse, new_pro, missing_con, better, case)
        
    # given a case and a case base, returns a list of best precedents as comparisons
    def find_precedents(self, cb):
        same_outcome = [case for case in cb.cases if case.outcome == self.outcome]
        comparisons = []
        for case in same_outcome:
            comparisons.append(self.compare(case))
            
        removals = []
        for c in comparisons:
            for o_c in comparisons:
                if set(c.differences) > set(o_c.differences):
                    removals.append(c)

        return [com for com in comparisons if com not in removals]
    
    # given a case c and another case c', checks whether c is as good for outcome(c') as c'
    def as_good(self, case):
        switch_case = Case(None, self.fs, self.opposite_outcome)
        com = switch_case.compare(case)
        
        if not com.differences:
            return True
        else:
            return False
        
    # Takes the focus case, a precedent and a dictionary with weights
    # Returns a comparison object comparing the focus case and the precedent
    def find_differences(self, other, weights):
            
        # Dictionaries consisting of {feature : difference (value focus - value pre)}
        # Worse are the negative differences: dif that make the focus case less likely to imitate the decision of pre
        # Better are the positive differences: dif that make the focus case more likely to imitate the decision of pre
        worse, better = {}, {}
        
        # The summed absolute weighted differences of worse and better
        total_worse, total_better = 0, 0

        for feature, value in other.fs.items():
            if feature.get_tendency(value) == other.outcome:
                if value > self.fs[feature]:
                    worse[feature] = value
                    total_worse += abs(weights[feature.name] * value)
                elif value < self.fs[feature]:
                    better[feature] = value
                    total_better += abs(weights[feature.name] * value)
            else:
                if value < self.fs[feature]:
                    worse[feature] = value
                    total_worse += abs(weights[feature.name] * value)
                elif value > self.fs[feature]:
                    better[feature] = value
                    total_better += abs(weights[feature.name] * value)
                
        return W_comparison(worse, total_worse, better, total_better, other)
         
        
class Feature():
    def __init__(self, name, tendency, factor):
        # a feature is defined as a triple Feature(name, tendency, factor)
        # tendency: 
        # ---- if it is a factor:
        # ------------ a dict representing the partial function Td
        # ---- else:
        # ------------ the outcome a higher value on the dict promotes (0 or 1)
        # factor: binary value representing whether it is a factor or dimension
        
        self.name = name
        self.tendency = tendency
        self.factor = factor
        
        # given a feature and a value, returns the tendency of that value
    def get_tendency(self, value):
        if self.factor:
            return self.tendency[value]
        else:
            return self.tendency
        
    def print_feature(self):
        print(self.name)
        
        
class Comparison():
    # this object is only used by AF-CBA
    def __init__(self, missing_pro, new_con, worse, new_pro, missing_con, better, case):
        # a comparison object consists of all differences between a case c and the focus case (relevant and other)
        # and c itself
        self.missing_pro = missing_pro
        self.new_con = new_con
        self.worse = worse
        
        self.new_pro = new_pro
        self.missing_con = missing_con
        self.better = better
        
        # all relevant differences
        self.differences = self.missing_pro + self.new_con + self.worse
        
        self.case = case
        
    def print_dif(self):
        for feature, value in self.missing_pro:
            print('MissingPro(' + feature.name + ')')
        for feature, value in self.new_con:
            print('NewCon(' + feature.name + ')')
        for feature, value in self.worse:
            print('Worse(' + feature.name + ')')
            
    def print_com(self):
        print('relevant differences:')
        self.print_dif()
        print('other differences:')
        for feature, value in self.new_pro:
            print('NewPro(' + feature.name + ')')
        for feature, value in self.missing_con:
            print('MissingCon(' + feature.name + ')')
        for feature, value in self.better:
            print('Better(' + feature.name + ')')
        print('------')
            
class CB():
    def __init__(self, cases):
        # a CB oject can simply be initalized as a list of cases
        self.cases = cases
        
        # we add the attribute length
        self.length = len(self.cases)
        
    def print_cb(self, n):
        for case in self.cases[0:n]:
            case.print_case()
            
    def split(self, p_test):
        # given a CB and a relative test size (0-1),
        # returns a test- and train CB
        test = CB(self.cases[:round(p_test * self.length)])
        train = CB(self.cases[round(p_test * self.length):])
        
        return test, train
            
    def check_consistency(self):
        # given a CB, returns whether it is consistent + a sorted dictionary with the inconsistency of cases
        other_cases = []
        consistent = True
        for case in self.cases:
            for other_case in self.cases:
                if case.outcome != other_case.outcome:
                    if other_case.as_good(case):
                        consistent = False
                        other_cases.append(other_case.name)
        
        freq = {}
        for item in other_cases:
            if item in freq:
                freq[item] += 1
            else:
                freq[item] = 1
        sort_freq = {k: v for k, v in sorted(freq.items(), key = lambda item: item[1], reverse = True)}
        
        return consistent, sort_freq
    
    def make_consistent(self, n_removals):
        # given a case base and the number of cases to remove per iteration
        # returns a consistent case base
        
        consistent, case_dict = self.check_consistency()

        while not consistent:
            removals = []
            for r in list(case_dict.keys())[0:n_removals]:
                for case in self.cases:
                    if case.name == r:
                        removals.append(case)

            for r in removals:           
                self.cases.remove(r)

            consistent, case_dict = self.check_consistency()
            
        return CB(self.cases)
                        
class W_comparison():
    # Weighted comparison object. This object is only used by the algorithms of Chapter 7.
    def __init__(self, worse, w_dif, better, b_dif, case):
        self.worse = worse
        self.w_dif = w_dif
        self.better = better
        self.b_dif = b_dif
        self.case = case
        self.balance = b_dif - w_dif
        
    def print_com(self):
        print('worse: ', self.worse)
        print('w_dif: ', self.w_dif)
        print('better: ', self.better)
        print('b_dif: ', self.b_dif)
        
